Nous allons détailler les fonctions dans ce notebook.

**IMPORT**

In [11]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from pathlib import Path
import os
# from webdriver_manager.chrome import ChromeDriverManager
# import chromedriver_binary

In [8]:
url_kindle_accueil = "https://www.amazon.com/b?node=154606011"
url_kindle_cat_art_and_photo ="https://www.amazon.com/s?bbn=154606011&rh=n%3A133140011%2Cn%3A154606011%2Cn%3A154607011&dc&qid=1648440703&rnid=154606011&ref=lp_154606011_nr_n_0"
url_kindle_cat_art_and_photo_subcat_music_p1 = "https://www.amazon.com/s?rh=n%3A156302011&fs=true&ref=lp_156302011_sar"
url_kindle_cat_art_and_photo_subcat_umusic_p2 = "https://www.amazon.com/s?i=digital-text&rh=n%3A156302011&fs=true&page=2&qid=1648440846&ref=sr_pg_2"
url_kindle_cat_art_and_photo_subcat_umusic_p3  ="https://www.amazon.com/s?i=digital-text&rh=n%3A156302011&fs=true&page=3&qid=1648440968&ref=sr_pg_3"

### video Python Selenium Web Scraping Tutorial: How to scrape Amazon with Python, Selenium and BeautifulSoup

In [9]:
from selenium.webdriver.chrome.service import Service



In [4]:
DRIVER_PATH_UBUNTU = str(Path("../chromedriver.exe").resolve()) #  permet de faire un chemin absolu ( méthoe resolve )
#pour le driver que l'on a mis dans le répertoire au dessus 

In [5]:
DRIVER_PATH_UBUNTU

'/home/thomas/code/thomasgassin/kindle-scrapper/chromedriver.exe'

In [6]:
# DRIVER_PATH_WINDOWS = str(

solution 1:
installer chrome 
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
and I did install sudo dpkg -i google-chrome-stable_current_amd64.deb

solution 2 : reprendre les cours du wagon et installer chrome_binary

```bash
pip install selenium
pip install chromedriver-binary==77.0.3865.40.0 # Version might be different!
```

OK! Now that this is done, open the `test_advanced_scraping.py` file and copy/paste the following code:

```python
from selenium import webdriver
import chromedriver_binary

driver = webdriver.Chrome()
driver.get("https://recipes.lewagon.com/recipes/advanced")

# driver.quit()
```

solution 3 : 
    le chemin pour executer chorme n'est pas le bon et il faut le modifier avec options

In [7]:
import chromedriver_binary

In [8]:
from selenium.webdriver.chrome.options import Options
options = Options()
options.binary_location = "C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"    #chrome binary location specified here
options.add_argument("--start-maximized") #open Browser in maximized mode
options.add_argument("--no-sandbox") #bypass OS security model
options.add_argument("--disable-dev-shm-usage") #overcome limited resource problems
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH_UBUNTU)
driver.get('url_kindle_accueil')

/tmp/ipykernel_12162/2941951217.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH_UBUNTU)


WebDriverException: Message: Can not connect to the Service /home/thomas/code/thomasgassin/kindle-scrapper/chromedriver.exe


In [12]:
driver = webdriver.Chrome()
driver.get("https://recipes.lewagon.com/recipes/advanced")

WebDriverException: Message: unknown error: Chrome failed to start: exited abnormally.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /opt/google/chrome/google-chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x55848b053ad3 <unknown>
#1 0x55848adb3568 <unknown>
#2 0x55848add6b37 <unknown>
#3 0x55848add223a <unknown>
#4 0x55848ae0d2ba <unknown>
#5 0x55848ae074e3 <unknown>
#6 0x55848addcd1a <unknown>
#7 0x55848addde75 <unknown>
#8 0x55848b081efd <unknown>
#9 0x55848b09b19b <unknown>
#10 0x55848b083c65 <unknown>
#11 0x55848b09bec8 <unknown>
#12 0x55848b077360 <unknown>
#13 0x55848b0b7a68 <unknown>
#14 0x55848b0b7be8 <unknown>
#15 0x55848b0d17fd <unknown>
#16 0x7fd6240bc609 <unknown>


In [ ]:
s = Service(DRIVER_PATH_UBUNTU) # solution 1 
driver = webdriver.Chrome(service=s) 

In [13]:
browser = webdriver.Chrome( executable_path = DRIVER_PATH ) # problume ne fonctionne pas 
# solution de stack overflow ajouter la ligne service

/tmp/ipykernel_3474/872224566.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome( executable_path=DRIVER_PATH)


WebDriverException: Message: Can not connect to the Service /home/thomas/code/thomasgassin/kindle-scrapper/chromedriver.exe


In [7]:
from selenium import webdriver
# from selenium.webdriver.chrome.options import Options

options = Options()
options.binary_location = "C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
driver = webdriver.Chrome(chrome_options = options, executable_path="C:/Users/Thomas/chromedriver.exe")
# driver.get('http://google.com/')
# print("Chrome Browser Invoked")
# driver.quit()

NameError: name 'Options' is not defined

In [33]:
driver = webdriver.Chrome()
driver.get("https://recipes.lewagon.com/recipes/advanced")


WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home


In [12]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Could not get version for google-chrome.Is google-chrome installed?
Get LATEST chromedriver version for None google-chrome
Driver [/home/thomas/.wdm/drivers/chromedriver/linux64/99.0.4844.51/chromedriver] found in cache
/tmp/ipykernel_18588/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x55bd1f1dc7d3 <unknown>
#1 0x55bd1ef38688 <unknown>
#2 0x55bd1ef59a38 <unknown>
#3 0x55bd1ef5768f <unknown>
#4 0x55bd1ef92355 <unknown>
#5 0x55bd1ef8c5a3 <unknown>
#6 0x55bd1ef61ddc <unknown>
#7 0x55bd1ef62de5 <unknown>
#8 0x55bd1f20d49d <unknown>
#9 0x55bd1f22660c <unknown>
#10 0x55bd1f20f205 <unknown>
#11 0x55bd1f226ee5 <unknown>
#12 0x55bd1f203070 <unknown>
#13 0x55bd1f242488 <unknown>
#14 0x55bd1f24260c <unknown>
#15 0x55bd1f25bc6d <unknown>
#16 0x7fa9af3ae609 <unknown>


In [13]:
browser = webdriver.Chrome(executable_path=driver_path)
browser.get(url)

NameError: name 'driver_path' is not defined

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

browser = webdriver.Firefox()

browser.get('http://www.yahoo.com')
assert 'Yahoo' in browser.title

elem = browser.find_element(By.NAME, 'p')  # Find the search box
elem.send_keys('seleniumhq' + Keys.RETURN)

browser.quit()

WebDriverException: Message: 'geckodriver' executable needs to be in PATH. 


### Suite à des problèmes ( je n'ai pas réussi à le fonctionner la version avec chrome c'est pourquoi je réessay avec la version de Firefox.

le code qui a fonctionné pour un déplacement du fichier avec le terminal
 sudo cp geckodriver.exe  /usr/local/bin/geckodriver.exe

In [45]:
from pathlib import Path
from selenium import webdriver
# DRIVER_PATH = str(Path('/usr/local/bin/geckodriver.exe').resolve())
DRIVER_PATH = str(Path('geckodriver.exe').resolve())

# print(DRIVER_PATH)
# browser = webdriver.Firefox(executable_path= DRIVER_PATH)
# browser.get(url_kindle_accueil)

/home/thomas/code/thomasgassin/kindle-scrapper/notebooks/geckodriver.exe


/tmp/ipykernel_1590/3895015387.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path= DRIVER_PATH)


WebDriverException: Message: Can not connect to the Service /home/thomas/code/thomasgassin/kindle-scrapper/notebooks/geckodriver.exe


In [47]:
import os
os.startfile("/home/thomas/code/thomasgassin/kindle-scrapper/notebooks/geckodriver.exe")

AttributeError: module 'os' has no attribute 'startfile'

# Trouver l'url des livres sur une page données

In [11]:
response = requests.get(url_kindle_cat_art_and_photo_subcat_umusic_p2)
soup = BeautifulSoup(response.content, "html.parser")

In [10]:
soup

<!--
        To discuss automated access to Amazon data please contact api-services-support@amazon.com.
        For information about migrating to our APIs refer to our Marketplace APIs at https://developer.amazonservices.com/ref=rm_5_sv, or our Product Advertising API at https://affiliate-program.amazon.com/gp/advertising/api/detail/main.html/ref=rm_5_ac for advertising use cases.
-->
<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<title>Sorry! Something went wrong!</title>
<style>
  html, body {
    padding: 0;
    margin: 0
  }

  img {
    border: 0
  }

  #a {
    background: #232f3e;
    padding: 11px 11px 11px 192px
  }

  #b {
    position: absolute;
    left: 22px;
    top: 12px
  }

  #c {
    position: relative;
    max-width: 800px;
    padding: 0 40px 0 0
  }

  #e, #f {
    height: 35px;
    border: 0;
    font-size: 1em
  

In [ ]:
books_html = soup.find_all("article", class_="product_pod")
len(books_html)

# Sur une page récupérer les données : de classement de vente , nom du livre catérgorie